In [1]:
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier, Pool
from imblearn.combine import SMOTEENN
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.base import BaseEstimator

from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import roc_auc_score, average_precision_score
from scipy.stats import randint, uniform

In [2]:
# ======================
# 1. 讀取資料
# ======================
train = pd.read_excel("train.xlsx")
test  = pd.read_excel("test.xlsx")

X = train.drop(columns=['崩塌'])
y = train['崩塌']

# ======================
# 2. 切分與補值
# ======================
X_train, X_val, y_train, y_val = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)
imputer = SimpleImputer(strategy='median')
X_train_imp = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_val_imp   = pd.DataFrame(imputer.transform(X_val),   columns=X_val.columns,   index=X_val.index)

In [4]:
# ======================
# 3. 特徵選擇（固定 6 + 從重要度補到總共 14）
# ======================
from catboost import CatBoostClassifier, Pool

DESIRED_K = 14
base_features = ['植生指標', '平均坡向', '最小曲率', '水系距', '斷層距', '順向坡指標']
feature_names = list(X_train_imp.columns)

# 先用一個小模型估特徵重要度
model0 = CatBoostClassifier(
    loss_function='Logloss', iterations=200, depth=6, learning_rate=0.1,
    l2_leaf_reg=3.0, bootstrap_type='Bayesian', bagging_temperature=1.0,
    random_seed=42, verbose=False, thread_count=-1, rsm=0.8, border_count=128
)
model0.fit(Pool(X_train_imp, y_train, feature_names=feature_names))
imp = model0.get_feature_importance(Pool(X_train_imp, y_train, feature_names=feature_names))
imp_df = (
    pd.DataFrame({'feature': feature_names, 'importance': imp})
      .sort_values('importance', ascending=False)
      .reset_index(drop=True)
)

# 依重要度排序的清單（先不含 base_features，以免重複）
imp_order = [f for f in imp_df['feature'].tolist() if f in X_train_imp.columns]

# 先放入「固定 6」且確保順序
final_features = [f for f in base_features if f in X_train_imp.columns]

# 再依重要度補入其餘特徵，直到總數達 14
for f in imp_order:
    if f not in final_features:
        final_features.append(f)
    if len(final_features) >= DESIRED_K:
        break

# 若欄位不足（極少見），直接用目前能找到的全部
if len(final_features) < DESIRED_K:
    print(f"[Warn] 可用欄位不足 {DESIRED_K}，目前僅 {len(final_features)} 個。")

print(f"\n>>> 最終特徵（{len(final_features)} 個；含固定 6）：{final_features}")

X_train_sel = X_train_imp[final_features].copy()
X_val_sel   = X_val_imp[final_features].copy()
def align(df, cols): return df.reindex(columns=cols, fill_value=0)

# ======================
# 4. 定義 SafeSMOTEENN（不再繼承 BaseSampler）
# ======================
from sklearn.base import BaseEstimator
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE

class SafeSMOTEENN(BaseEstimator):
    """先用 SMOTEENN；若只剩單一類，退回純 SMOTE 以保證兩類皆存在。"""
    def __init__(self, random_state=42, sampling_strategy=0.9):
        self.random_state = random_state
        self.sampling_strategy = sampling_strategy
        self._smoteenn = SMOTEENN(random_state=random_state,
                                  sampling_strategy=sampling_strategy)

    def fit(self, X, y):
        return self

    def fit_resample(self, X, y):
        X_res, y_res = self._smoteenn.fit_resample(X, y)
        # 若被 ENN 清到只剩一類，退回 SMOTE
        if len(np.unique(y_res)) < 2:
            smote = SMOTE(random_state=self.random_state)
            X_res, y_res = smote.fit_resample(X, y)
        return X_res, y_res

# ======================
# 5. NaN / Inf 防護
# ======================
def _nan_guard(X):
    X = np.asarray(X, dtype=float)
    return np.nan_to_num(X, nan=0.0, posinf=1e12, neginf=-1e12)

# ======================
# 6. Pipeline 與搜尋空間（以 PR-AUC 作為 refit 目標）
# ======================
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import FunctionTransformer
from imblearn.pipeline import Pipeline as ImbPipeline
from scipy.stats import randint, uniform

cv = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)

base = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='PRAUC',          # 以 PR-AUC 評估
    verbose=False,
    thread_count=-1,
    bootstrap_type='Bernoulli',
    subsample=0.8,
    auto_class_weights=None       # 關閉自動類別權重，避免除以 0
)

pipeline = ImbPipeline(steps=[
    ('resample', SafeSMOTEENN(random_state=42, sampling_strategy=0.9)),
    ('nanfix', FunctionTransformer(_nan_guard, validate=False)),
    ('clf', base)
])

param_dist = {
    'clf__iterations':      randint(1200, 2000),
    'clf__depth':           randint(5, 8),
    'clf__learning_rate':   uniform(0.025, 0.02),
    'clf__l2_leaf_reg':     uniform(4, 8),
    'clf__rsm':             uniform(0.7, 0.2),
    'clf__random_strength': randint(1, 6),
    'clf__border_count':    randint(128, 255),
}

# 同時計算 ROC-AUC 與 PR-AUC，但「以 PR-AUC」決定最佳模型
scorers = {'roc_auc': 'roc_auc', 'prauc': 'average_precision'}

rs = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=60,
    scoring=scorers,
    refit='prauc',           # ✅ 以 PR-AUC 最高者為 best_estimator_
    cv=cv,
    n_jobs=-1,
    verbose=2,
    random_state=42,
    error_score=np.nan
)

# ======================
# 7. 開始搜尋
# ======================
print("🔎 RandomizedSearchCV（含折內重採樣）開始...")
rs.fit(X_train_sel, y_train)
print("✅ 最佳參數:", rs.best_params_)

best_prauc = rs.cv_results_['mean_test_prauc'][rs.best_index_]
best_roc   = rs.cv_results_['mean_test_roc_auc'][rs.best_index_]
print(f"✅ 交叉驗證最佳 PR-AUC: {best_prauc:.4f}")
print(f"✅ 對應 ROC-AUC      : {best_roc:.4f}")

best_pipeline = rs.best_estimator_



>>> 最終特徵（14 個；含固定 6）：['植生指標', '平均坡向', '最小曲率', '水系距', '斷層距', '順向坡指標', 'f_85', 'f_32', 'f_48', 'f_128', 'f_199', 'f_109', 'f_52', 'f_156']
🔎 RandomizedSearchCV（含折內重採樣）開始...
Fitting 7 folds for each of 60 candidates, totalling 420 fits
[CV] END clf__border_count=230, clf__depth=5, clf__iterations=1470, clf__l2_leaf_reg=9.85595153449124, clf__learning_rate=0.03697316968394074, clf__random_strength=2, clf__rsm=0.7311989040672405; total time=  24.8s
[CV] END clf__border_count=230, clf__depth=5, clf__iterations=1470, clf__l2_leaf_reg=9.85595153449124, clf__learning_rate=0.03697316968394074, clf__random_strength=2, clf__rsm=0.7311989040672405; total time=  25.8s
[CV] END clf__border_count=230, clf__depth=5, clf__iterations=1470, clf__l2_leaf_reg=9.85595153449124, clf__learning_rate=0.03697316968394074, clf__random_strength=2, clf__rsm=0.7311989040672405; total time=  28.1s[CV] END clf__border_count=230, clf__depth=5, clf__iterations=1470, clf__l2_leaf_reg=9.85595153449124, clf__learning_rate=

In [10]:
# ======================
# 8. 驗證集評估（先用 best_pipeline 做 baseline）
# ======================
from sklearn.metrics import (
    accuracy_score,        
    f1_score
)

best_pipeline.fit(X_train_sel, y_train)
X_val_sel = align(X_val_sel, X_train_sel.columns)
y_prob_val = best_pipeline.predict_proba(X_val_sel)[:, 1]

val_roc   = roc_auc_score(y_val, y_prob_val)
val_prauc = average_precision_score(y_val, y_prob_val)
print(f"📈 驗證集 ROC-AUC: {val_roc:.4f}")
print(f"📈 驗證集 PR-AUC : {val_prauc:.4f}")

# 小工具：分數分佈摘要
def summarize_scores(arr, name):
    a = np.asarray(arr)
    p = np.percentile(a, [0, 25, 50, 75, 100])
    print(f"{name}: min={p[0]:.4f} p25={p[1]:.4f} p50={p[2]:.4f} p75={p[3]:.4f} max={p[4]:.4f} mean={a.mean():.4f}")

summarize_scores(y_prob_val, "VAL baseline probs")

# ======================
# 9. Bagging（support：prob-avg / rank-avg / weighted-avg + early stopping）
# ======================
best_cb: CatBoostClassifier = best_pipeline.named_steps['clf']
cb_params = best_cb.get_params()

resampler = SafeSMOTEENN(random_state=42, sampling_strategy=0.9)
X_res_bag, y_res_bag = resampler.fit_resample(X_train_sel, y_train)

# 讓子模型多樣化：不同 random_seed + CatBoost 的隨機性
seeds = [11, 22, 33, 44, 55]
bag_models = []
val_each = []   # 儲存各模型在驗證集上的機率（做加權用）
weights  = []   # 儲存各模型在驗證集 PR-AUC（當權重）

for sd in seeds:
    p = cb_params.copy()
    p['random_seed'] = sd
    p['verbose'] = False

    # ⭐ 用 early_stopping：eval_set=原始驗證集，避免重採樣過擬合
    m = CatBoostClassifier(**p)
    m.fit(
        X_res_bag, y_res_bag,
        eval_set=(X_val_sel, y_val),
        use_best_model=True,
        verbose=False
    )
    bag_models.append(m)

    vprob = m.predict_proba(X_val_sel)[:, 1]
    val_each.append(vprob)
    weights.append(average_precision_score(y_val, vprob))

val_scores = np.vstack(val_each)

# === (A) 機率平均 ===
probs_val_mean = val_scores.mean(axis=0)

# === (B) 秩次平均 ===
ranks = np.argsort(np.argsort(val_scores, axis=1), axis=1) + 1
avg_rank = ranks.mean(axis=0)
probs_val_rankavg = (avg_rank - avg_rank.min()) / (avg_rank.max() - avg_rank.min())

# === (C) 加權平均（以各子模型的驗證 PR-AUC 為權重） ===
w = np.asarray(weights, dtype=float)
w = np.maximum(w, 1e-9)
w = w / w.sum()
probs_val_wavg = np.average(val_scores, axis=0, weights=w)

for name, s in {
    "Prob-Avg": probs_val_mean,
    "Rank-Avg": probs_val_rankavg,
    "W-Prob-Avg": probs_val_wavg
}.items():
    roc = roc_auc_score(y_val, s)
    pr  = average_precision_score(y_val, s)
    print(f"🤝 Bagging[{name}] ROC-AUC={roc:.4f} | PR-AUC={pr:.4f}")

# 以驗證集 PR-AUC 表現最好者作為「最終集成策略」
cand = {
    "Prob-Avg": probs_val_mean,
    "Rank-Avg": probs_val_rankavg,
    "W-Prob-Avg": probs_val_wavg
}
best_name, best_val_scores = max(
    cand.items(), key=lambda kv: average_precision_score(y_val, kv[1])
)
print(f"✅ 以驗證集 PR-AUC 選擇的 Bagging 策略：{best_name}")

summarize_scores(best_val_scores, f"VAL {best_name} probs")


📈 驗證集 ROC-AUC: 0.8681
📈 驗證集 PR-AUC : 0.5744
VAL baseline probs: min=0.0000 p25=0.0022 p50=0.0263 p75=0.3099 max=0.9998 mean=0.2126
🤝 Bagging[Prob-Avg] ROC-AUC=0.8701 | PR-AUC=0.5801
🤝 Bagging[Rank-Avg] ROC-AUC=0.8702 | PR-AUC=0.5816
🤝 Bagging[W-Prob-Avg] ROC-AUC=0.8701 | PR-AUC=0.5801
✅ 以驗證集 PR-AUC 選擇的 Bagging 策略：Rank-Avg
VAL Rank-Avg probs: min=0.0000 p25=0.2515 p50=0.4963 p75=0.7468 max=1.0000 mean=0.4994


In [21]:
THRESHOLD = 0.504
y_val_pred_fixed = (best_val_scores >= THRESHOLD).astype(int)
print(
    f"🧪 VAL @ thr={THRESHOLD:.2f} | "
    f"F1={f1_score(y_val, y_val_pred_fixed, zero_division=0):.4f} | "
    f"Acc={accuracy_score(y_val, y_val_pred_fixed):.4f}"
)

# ======================
# 10. 產生 submission（沿用最佳策略 + 固定門檻 0.02）
# ======================
X_test_imp = pd.DataFrame(imputer.transform(test), columns=test.columns, index=test.index)
X_test_sel = align(X_test_imp[final_features].copy(), X_train_sel.columns)

test_scores = np.vstack([m.predict_proba(X_test_sel)[:, 1] for m in bag_models])

probs_test_mean   = test_scores.mean(axis=0)
ranks_test        = np.argsort(np.argsort(test_scores, axis=1), axis=1) + 1
avg_rank_test     = ranks_test.mean(axis=0)
probs_test_rank   = (avg_rank_test - avg_rank_test.min()) / (avg_rank_test.max() - avg_rank_test.min())
probs_test_wavg   = np.average(test_scores, axis=0, weights=w)

probs_test_map = {
    "Prob-Avg":   probs_test_mean,
    "Rank-Avg":   probs_test_rank,
    "W-Prob-Avg": probs_test_wavg
}
probs_test_final = probs_test_map[best_name]

# ---- 固定門檻 ----
y_pred_test = (probs_test_final >= THRESHOLD).astype(int)
print(f"✂️ 使用固定門檻 threshold={THRESHOLD:.4f}")

# === 0/1 統計 ===
from collections import Counter
cnt = Counter(y_pred_test.tolist())
n0 = cnt.get(0, 0); n1 = cnt.get(1, 0); n = len(y_pred_test)
print(f"📊 Test 預測分佈：0={n0} | 1={n1} | 總數={n} | 正例比例={n1/n:.4f}")

# 輸出
id_col = 'ID' if 'ID' in test.columns else None
submission = pd.DataFrame({
    "ID": test[id_col] if id_col else range(len(test)),
    "Label": y_pred_test
})
submission.to_csv("submission.csv", index=False)
print(f"✅ submission.csv 已輸出（Bagging-{best_name} + thr={THRESHOLD}）; 筆數：{len(submission)}")

🧪 VAL @ thr=0.50 | F1=0.3654 | Acc=0.6067
✂️ 使用固定門檻 threshold=0.5040
📊 Test 預測分佈：0=500 | 1=500 | 總數=1000 | 正例比例=0.5000
✅ submission.csv 已輸出（Bagging-Rank-Avg + thr=0.504）; 筆數：1000


In [ ]:
# # ======================
# # 8. 驗證集評估
# # ======================
# best_pipeline.fit(X_train_sel, y_train)
# X_val_sel = align(X_val_sel, X_train_sel.columns)
# y_prob_val = best_pipeline.predict_proba(X_val_sel)[:, 1]

# val_roc   = roc_auc_score(y_val, y_prob_val)
# val_prauc = average_precision_score(y_val, y_prob_val)
# print(f"📈 驗證集 ROC-AUC: {val_roc:.4f}")
# print(f"📈 驗證集 PR-AUC : {val_prauc:.4f}")

# # ======================
# # 9. Bagging（rank averaging）
# # ======================
# best_cb: CatBoostClassifier = best_pipeline.named_steps['clf']
# cb_params = best_cb.get_params()

# resampler = SafeSMOTEENN(random_state=42, sampling_strategy=0.9)
# X_res_bag, y_res_bag = resampler.fit_resample(X_train_sel, y_train)

# seeds = [11, 22, 33, 44, 55]
# bag_models = []
# for sd in seeds:
#     p = cb_params.copy()
#     p['random_seed'] = sd
#     p['verbose'] = False
#     m = CatBoostClassifier(**p)
#     m.fit(X_res_bag, y_res_bag)
#     bag_models.append(m)

# val_scores = np.vstack([m.predict_proba(X_val_sel)[:, 1] for m in bag_models])
# ranks = np.argsort(np.argsort(val_scores, axis=1), axis=1) + 1
# avg_rank = ranks.mean(axis=0)
# probs_val_rankavg = (avg_rank - avg_rank.min()) / (avg_rank.max() - avg_rank.min())

# val_roc_bag   = roc_auc_score(y_val, probs_val_rankavg)
# val_prauc_bag = average_precision_score(y_val, probs_val_rankavg)
# print(f"🤝 Bagging(秩次平均) ROC-AUC: {val_roc_bag:.4f}")
# print(f"🤝 Bagging(秩次平均) PR-AUC : {val_prauc_bag:.4f}")

# # ======================
# # 10. 產生 submission
# # ======================
# X_test_imp = pd.DataFrame(imputer.transform(test), columns=test.columns, index=test.index)
# X_test_sel = align(X_test_imp[final_features].copy(), X_train_sel.columns)

# test_scores = np.vstack([m.predict_proba(X_test_sel)[:, 1] for m in bag_models])
# ranks_test = np.argsort(np.argsort(test_scores, axis=1), axis=1) + 1
# avg_rank_test = ranks_test.mean(axis=0)
# probs_test_rankavg = (avg_rank_test - avg_rank_test.min()) / (avg_rank_test.max() - avg_rank_test.min())
# y_pred_test = (probs_test_rankavg > 0.5).astype(int)

# # === 新增：統計 0/1 數量與比例 ===
# from collections import Counter
# cnt = Counter(y_pred_test.tolist())
# n0 = cnt.get(0, 0)
# n1 = cnt.get(1, 0)
# n  = len(y_pred_test)
# print(f"📊 Test 預測分佈：0={n0} | 1={n1} | 總數={n} | 正例比例={n1/n:.4f}")

# id_col = 'ID' if 'ID' in test.columns else None
# submission = pd.DataFrame({
#     "ID": test[id_col] if id_col else range(len(test)),
#     "Label": y_pred_test
# })
# submission.to_csv("submission.csv", index=False)
# print("✅ submission.csv 已輸出（Bagging-秩次平均 + 0.5 閾值）; 筆數：", len(submission))


📈 驗證集 ROC-AUC: 0.8681
📈 驗證集 PR-AUC : 0.5744
🤝 Bagging(秩次平均) ROC-AUC: 0.8700
🤝 Bagging(秩次平均) PR-AUC : 0.5797
📊 Test 預測分佈：0=495 | 1=505 | 總數=1000 | 正例比例=0.5050
✅ submission.csv 已輸出（Bagging-秩次平均 + 0.5 閾值）; 筆數： 1000
